In [2]:
import pandas as pd
from scipy import stats
import pingouin

### まずデータを読み込む

In [3]:
df =pd.read_csv("data.csv")
df.head()

,filename,ID,age,sex,LR,AMD,RVO,Gla,MH,DR,RD,RP,AO,DM
0,000000_00.jpg,3090,78,M,L,0,0,0,0,0,0,0,0,0
1,000000_01.jpg,3090,78,M,R,0,0,0,0,0,0,0,0,0
2,000001_00.jpg,2702,76,F,L,0,0,0,0,0,0,0,0,0
3,000001_01.jpg,2702,76,F,R,0,0,0,0,0,0,0,0,0
4,000001_02.jpg,1136,69,M,L,0,0,1,0,0,0,0,0,0


# 2クラスの平均比較
Welchのt検定を行う<br>
2クラスの分散が等しいことを仮定しなくてよいので、非常に有用。全部これでやる。

### AMD群とRVO群の年齢を比較する

In [4]:
# データの読み込み
age0 = df[df["AMD"] == 1]["age"]
age1 = df[df["RVO"] == 1]["age"]

In [5]:
# Welchのt検定
stats.ttest_ind(age0, age1, equal_var = False)

Ttest_indResult(statistic=13.674213613306113, pvalue=2.3130943335506705e-39)

### テンプレート化
2クラス間の平均値やｐ値などよく使うのは、この関数で十分

In [6]:
# よくある平均±SD, p値を求める
def compare_ave_2group(val0, val1, digit = 1, digit_p = 3):
    mean0 = round(val0.mean(), digit)
    std0 = round(val0.std(), digit)
    mean1 = round(val1.mean(), digit)
    std1 = round(val1.std(), digit)
    p_value = round(stats.ttest_ind(val0, val1, equal_var = False)[1], digit_p)
    return mean0, std0, mean1, std1, p_value

In [7]:
compare_ave_2group(age0, age1)

(75.3, 8.9, 66.8, 12.2, 0.0)

# 3クラスの平均比較
WelchのANOVAを行う<br>
3クラスの分散が等しいことを仮定しなくてよいので、非常に有用。全部これでやる。

### AMD群とRVO, DR群の年齢を比較する

In [17]:
# 病名の列と、検定したい値の列を並べる。（pingionの書体に合わせる）
df_amd = pd.DataFrame(columns=["age", "des"])
df_rvo = pd.DataFrame(columns=["age", "des"])
df_dr = pd.DataFrame(columns=["age", "des"])
df_amd["age"] = df[df["AMD"] == 1]["age"]
df_amd["des"] = "AMD"
df_rvo["age"] = df[df["RVO"] == 1]["age"]
df_rvo["des"] = "RVO"
df_dr["age"] = df[df["DR"] == 1]["age"]
df_dr["des"] = "DR"
dfs = pd.concat([df_amd, df_rvo, df_dr])
dfs.head()

,age,des
11,88,AMD
50,77,AMD
62,72,AMD
65,73,AMD
76,69,AMD


### WelchのANOVAを行う
welch_ANOVAのoutputなどの内容は<br>
https://pingouin-stats.org/generated/pingouin.welch_anova.html#pingouin.welch_anova

In [18]:
aov = pingouin.welch_anova(dv='age', between='des', data=dfs)
aov

,Source,ddof1,ddof2,F,p-unc
0,des,2,992.771,280.043,3.639707e-97


### Games-Howell法にて個別検定
WelchのANOVAにて有意差を認めた（つまり、全体として差異がある）場合は、次は個別検定を行う<br>
個別検定は、Games-Howell法でいい。<br>
<br>
Gameshowellのoutputなどの内容は<br>
https://pingouin-stats.org/generated/pingouin.pairwise_gameshowell.html#pingouin.pairwise_gameshowell

In [21]:
pingouin.pairwise_gameshowell(data=dfs, dv="age", between='des')  

,A,B,T,df,diff,hedges,mean(A),mean(B),pval,se,tail
0,AMD,DR,23.600,611.941,11.459,1.231,75.327,63.868,0.001,0.343,two-sided
1,AMD,RVO,13.674,1074.948,8.480,0.832,75.327,66.847,0.001,0.439,two-sided
2,DR,RVO,-6.151,1150.243,-2.979,-0.245,63.868,66.847,0.001,0.342,two-sided


### テンプレート化
よく使うであろう多クラス間の平均など、ｐ値を算出

In [41]:
# よくある平均±SD, p値を求める
# tagと値の配列のペアとなっているdictionaryが投入される
# 全体で有意差があったか、タグと(平均, SD)のセット、ANOVAの結果のDataFrame, Games-Howellの結果のDataFrameが出力
def compare_ave_multi(dics, digit = 1):
    df_val = pd.DataFrame()
    vals = {}
    for key in dics.keys():
        dfs = pd.DataFrame()
        dfs["val"] = dics[key]
        dfs["tag"] = key
        df_val = df_val.append(dfs)
        vals[key] = (round(dics[key].mean(), digit), round(dics[key].std(), digit))
    aov = pingouin.welch_anova(dv='val', between='tag', data=df_val)
    anova_p = aov["p-unc"][0]
    if anova_p > 0.05:
        return False, vals,  [aov, None]
    else:
        ghs = pingouin.pairwise_gameshowell(data=df_val, dv="val", between='tag')  
        return True, vals, [aov, ghs]

### 使用例

In [45]:
df =pd.read_csv("data.csv")
digit = 1
dics = {}
tags = ["AMD", "RVO", "DR"]
for tag in tags:
    dics[tag] = df[df[tag] == 1]["age"]
res = compare_ave_multi(dics)

In [46]:
res[0], res[1]

(True, {'AMD': (75.3, 8.9), 'DR': (63.9, 11.9), 'RVO': (66.8, 12.2)})

In [48]:
res[2][0]

,Source,ddof1,ddof2,F,p-unc
0,tag,2,992.771,280.043,3.639707e-97


In [50]:
res[2][1]

,A,B,T,df,diff,hedges,mean(A),mean(B),pval,se,tail
0,AMD,DR,23.600,611.941,11.459,1.231,75.327,63.868,0.001,0.343,two-sided
1,AMD,RVO,13.674,1074.948,8.480,0.832,75.327,66.847,0.001,0.439,two-sided
2,DR,RVO,-6.151,1150.243,-2.979,-0.245,63.868,66.847,0.001,0.342,two-sided
